## Deliverable 2. Create a Customer Travel Destinations Map.

In [18]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,0,Norrtalje,SE,59.7580,18.7050,66.02,86,27,5.32,scattered clouds,2022-07-21 01:24:59
1,1,Rikitea,PF,-23.1203,-134.9692,73.56,76,0,5.61,light rain,2022-07-21 01:23:12
2,2,Cherskiy,RU,68.7500,161.3000,47.80,39,8,18.19,clear sky,2022-07-21 01:24:59
3,3,Laguna,US,38.4210,-121.4238,106.32,18,0,13.80,clear sky,2022-07-21 01:15:36
4,4,Ushuaia,AR,-54.8000,-68.3000,29.86,93,100,17.27,snow,2022-07-21 01:25:00


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for your trip? "))
max_temp = float(input("What is the maximum temperature for your trip? "))

What is the minimum temperature for your trip? 75
What is the maximum temperature for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matching_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
matching_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
5,5,Hilo,US,19.7297,-155.0900,84.27,85,41,5.01,scattered clouds,2022-07-21 01:13:57
21,21,Kavieng,PG,-2.5744,150.7967,83.08,75,96,11.99,light rain,2022-07-21 01:25:03
25,25,Najran,SA,17.4924,44.1277,84.85,51,100,7.83,overcast clouds,2022-07-21 01:25:05
26,26,Hithadhoo,MV,-0.6000,73.0833,79.99,77,98,6.20,overcast clouds,2022-07-21 01:25:05
27,27,Coari,BR,-4.0850,-63.1414,77.29,77,69,2.66,moderate rain,2022-07-21 01:25:05
28,28,Kahului,US,20.8947,-156.4700,83.97,54,40,25.32,scattered clouds,2022-07-21 01:22:20
31,31,Nichinan,JP,31.6000,131.3667,87.48,74,100,12.73,overcast clouds,2022-07-21 01:25:06
35,35,Jamestown,US,42.0970,-79.2353,82.02,66,0,9.22,clear sky,2022-07-21 01:12:25
36,36,Mecca,SA,21.4267,39.8261,86.05,44,86,6.22,overcast clouds,2022-07-21 01:25:08
41,41,Albany,US,42.6001,-73.9662,86.65,64,34,3.74,scattered clouds,2022-07-21 01:20:15


In [10]:
# 4a. Determine if there are any empty rows.
matching_cities_df.count()

City_ID        239
City           239
Country        239
Lat            239
Lng            239
Max Temp       239
Humidity       239
Cloudiness     239
Wind Speed     239
Description    239
Date           239
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = matching_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
5,Hilo,US,84.27,scattered clouds,19.7297,-155.0900,
21,Kavieng,PG,83.08,light rain,-2.5744,150.7967,
25,Najran,SA,84.85,overcast clouds,17.4924,44.1277,
26,Hithadhoo,MV,79.99,overcast clouds,-0.6000,73.0833,
27,Coari,BR,77.29,moderate rain,-4.0850,-63.1414,
28,Kahului,US,83.97,scattered clouds,20.8947,-156.4700,
31,Nichinan,JP,87.48,overcast clouds,31.6000,131.3667,
35,Jamestown,US,82.02,clear sky,42.0970,-79.2353,
36,Mecca,SA,86.05,overcast clouds,21.4267,39.8261,
41,Albany,US,86.65,scattered clouds,42.6001,-73.9662,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.  Skipping city") 
        hotel_df.loc[index, "Hotel Name"] = np.nan
hotel_df.head(10)

Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city
Hotel not found.  Skipping city


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
5,Hilo,US,84.27,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
21,Kavieng,PG,83.08,light rain,-2.5744,150.7967,Nusa Island Retreat
25,Najran,SA,84.85,overcast clouds,17.4924,44.1277,محمد هادي ال فطيح
26,Hithadhoo,MV,79.99,overcast clouds,-0.6000,73.0833,Scoop Guest House
27,Coari,BR,77.29,moderate rain,-4.0850,-63.1414,Dellrey Hotel Center
28,Kahului,US,83.97,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
31,Nichinan,JP,87.48,overcast clouds,31.6000,131.3667,Guest House Aki
35,Jamestown,US,82.02,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
36,Mecca,SA,86.05,overcast clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
41,Albany,US,86.65,scattered clouds,42.6001,-73.9662,NaN


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
5,Hilo,US,84.27,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
21,Kavieng,PG,83.08,light rain,-2.5744,150.7967,Nusa Island Retreat
25,Najran,SA,84.85,overcast clouds,17.4924,44.1277,محمد هادي ال فطيح
26,Hithadhoo,MV,79.99,overcast clouds,-0.6000,73.0833,Scoop Guest House
27,Coari,BR,77.29,moderate rain,-4.0850,-63.1414,Dellrey Hotel Center
28,Kahului,US,83.97,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
31,Nichinan,JP,87.48,overcast clouds,31.6000,131.3667,Guest House Aki
35,Jamestown,US,82.02,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
36,Mecca,SA,86.05,overcast clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
43,Spearfish,US,88.09,clear sky,44.4908,-103.8594,Travelodge by Wyndham Spearfish


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))